In [1]:
import tkinter as tk
import random, time

ventana = tk.Tk()
ventana.title("Busca minas y no de las que te gustan")
ventana.geometry("430x300")
ventana.iconbitmap("buscaminas meme.jpeg")
grilla  = tk.Frame(ventana)
grilla.pack(pady=10)

for i in range(10):
    for j in range(20):
        boton = tk.Button(grilla, text= " ")
        boton.grid(column=j, row = i)

etiqueta_cronometro = tk.Label(ventana, text = "Cronometro" , font = 10)
etiqueta_cronometro.pack()
inicio = time.time()

def cronometro():
    tiempo_transcurrido = time.time() - inicio
    string_tiempo = time.strftime('%M:%S', time.gmtime(tiempo_transcurrido))
    etiqueta_cronometro.config(text = f"Tiempo {string_tiempo}")
    ventana.after(10, cronometro)
cronometro()
ventana.mainloop()
